校验数据集

In [1]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.patches as patches
import cmaps
import cartopy.feature as cfeature
from datetime import datetime,timedelta
from matplotlib.offsetbox import AnchoredText#关于修改text位置的
import cv2
from pygac import get_reader_class
import ast
from Five_utiles4 import *
from tqdm import tqdm
proj =ccrs.NorthPolarStereo(central_longitude=17.5)
from sklearn.neighbors import BallTree
from shapely.geometry import Point, Polygon,LineString
#经过实验，按经纬度插值的清晰度小于先转化为实际地理坐标系再插值的清晰度
import concurrent.futures
import pyproj
import matplotlib.style as mplstyle
from matplotlib.colors import BoundaryNorm
from datetime import datetime
import glob
from shapely.geometry import Polygon, Point

In [2]:
def calculate_meantime(filepath,tle_name,central_lon,central_lat):
    #计算中心点50KM内的平均扫描时间
    tle_dir = r'Z:\NOAA_TLE'
    reader_cls = get_reader_class(filepath)
    reader = reader_cls(tle_dir=tle_dir, tle_name=tle_name)
    reader.read(filepath)
    times = reader.get_times()
    lons, lats = reader.get_lonlat()
    dis_array = geodistance(central_lon,central_lat,lons,lats)
    scantime150 = times[(dis_array<50).sum(axis=1)>0]
    #去除空值
    scantime150 = scantime150[~np.isnan(scantime150)]
    if len(scantime150)==0:
        return np.nan
    meantime = pd.Series(scantime150).mean()
    #规整到分钟
    return pd.to_datetime( meantime.strftime('%Y%m%d_%H%M'),format='%Y%m%d_%H%M')

In [3]:
#将json存储为yolo格式的标签
def json_to_yolo_obb_pose(load_dict):
    class_name = {'comma':0,'spiral':1}
    #由于都是正方形所有只需要读图片高度即可
    #标注的框
    shapes = load_dict['shapes']
    boxes,points = [],[]
    box_column  = ['group_id','label','x1','y1','x2','y2','x3','y3','x4','y4']
    point_column = ['group_id','x0','y0','is_visible']
    for idx,temp in enumerate(shapes):
        if temp['shape_type']=='point':
            points.append([temp['group_id']]+np.round(np.array(temp['points']).flatten(),3).tolist()+[2])
        else: 
            boxes.append([temp['group_id'],temp['label']]+np.round(np.array(temp['points']).flatten(),3).tolist())
    box_df = pd.DataFrame(boxes,columns=box_column).set_index('group_id')
    point_df = pd.DataFrame(points,columns=point_column).set_index('group_id')
    #检验标签的group_id
    flag = np.array_equal(np.sort(box_df.index.values),np.sort(point_df.index.values))
    if np.sum(~np.isin(box_df.label,['comma','spiral']))!=0:
        flag=False
    if flag:
        #当两者没有共同group_id时会为空
        result = pd.DataFrame(boxes,columns=box_column).set_index('group_id').join(pd.DataFrame(points,columns=point_column).set_index('group_id'),how='inner')
    else:
        print('%s:error'%load_dict['imagePath'])
        return []
    return result

In [4]:
chosed_columns = ['point_index', 'time','lon', 'lat','split_path_index', 'filepath','tle_name', 'dataset_type', 'satellite_name','filename','mean_time']
ERA5_matched = pd.read_excel(r"D:\Desktop\IMPMCT\new_ERA5_matched.xlsx",usecols=chosed_columns).reset_index(drop=True)
folders = os.listdir(r'D:\Desktop\IMPMCT\tracks')
json_forder = r'D:\Desktop\IMPMCT\cloud_jsons'

In [5]:
ID = 1
tracks,clouds = [],[]
for folder in tqdm(folders):
    #读取track
    xlsx_files = glob.glob(os.path.join(r'D:\Desktop\IMPMCT\tracks',folder, "*.xlsx"))
    if len(xlsx_files)!=1:
        print(folder)
        continue
    track_path = xlsx_files[0]
    #当轨迹是新轨迹时
    if os.path.basename(track_path)[0]=='n':
        track = pd.read_excel(track_path,header=None)
        track.columns = ['point_index','time','path_index','peak_value','lon','lat','split_path_index','relative']
    else:
        track = pd.read_excel(track_path)
    track['ID'] = ID
    tracks.append(track)
    #读取clouds文件
    infos = []
    cloud_folder = os.path.join(r'D:\Desktop\IMPMCT\core',folder)
    for file in os.listdir(cloud_folder):
        #避免选到json文件
        if file[-1]=='n':
            continue
        json_path = os.path.join(json_forder,file.replace('.png','.json'))
        with open(json_path,'r', encoding='UTF-8') as f:    
            load_dict = json.load(f)
        info = json_to_yolo_obb_pose(load_dict)
        info['filename'] = file
        infos.append(info)
    cloud_info = pd.concat(infos,axis=0)
    cloud_info['ID'] = ID
    clouds.append(cloud_info)
    ID+=1 

100%|██████████| 1185/1185 [00:48<00:00, 24.65it/s]


In [6]:
def geo2lonlat(central_lon,central_lat,distance_x,distance_y):
    #位移不变的经纬度转换
    earth_radius = 6370856#m
    lats = distance_y/(2*np.pi*earth_radius)*360+central_lat
    lon_diff = distance_x/(2*np.pi*earth_radius*np.cos(central_lat/180*np.pi))*360
    return central_lon+lon_diff,lats

In [7]:
ERA5_track,cloud_infos = pd.concat(tracks,axis=0),pd.concat(clouds,axis=0)
cloud_infos = pd.merge(cloud_infos,ERA5_matched,on=['filename'],how='left')
kpt_lon,kpt_lat = geo2lonlat(cloud_infos.lon,cloud_infos.lat,(cloud_infos.x0-401)*2e3,(401-cloud_infos.y0)*2e3)
cloud_infos['cloud_lon'],cloud_infos['cloud_lat'] = kpt_lon,kpt_lat

In [8]:
#判断cloud轨迹与track轨迹的重合度
for chosed_ID in tqdm(ERA5_track.ID.unique()):
    chosed_track = ERA5_track[ERA5_track['ID']==chosed_ID]
    chosed_cloud_info = cloud_infos[cloud_infos['ID']==chosed_ID]
    track_point_index = np.unique(chosed_track.point_index)
    cloud_point_index = np.unique(chosed_cloud_info.point_index)
    #cloud_info中的点在track中的比例
    more_num = len(cloud_point_index)-np.isin(cloud_point_index,track_point_index).sum()
    if more_num>3:
        print('轨迹%d多出%d个点'%(np.unique(chosed_track.path_index[0]),more_num))
    #粗略计算重叠点的平均距离
    overlap_cloud = chosed_cloud_info.loc[np.isin(chosed_cloud_info.point_index,track_point_index)]
    overlap_cloud['dis'] = geodistance(overlap_cloud.lon,overlap_cloud.lat,overlap_cloud.cloud_lon,overlap_cloud.cloud_lat)
    mean_dis = overlap_cloud.dis.mean()
    if mean_dis>150:
        print('轨迹%d平均距离%d'%(np.unique(chosed_track.path_index[0]),mean_dis))

 28%|██▊       | 327/1185 [00:00<00:01, 531.27it/s]

轨迹16926多出4个点
轨迹185平均距离194


 61%|██████▏   | 728/1185 [00:01<00:00, 559.35it/s]

轨迹27631多出4个点


 81%|████████  | 957/1185 [00:01<00:00, 556.87it/s]

轨迹40614多出6个点
轨迹42934平均距离150
轨迹442多出4个点
轨迹507多出4个点


100%|██████████| 1185/1185 [00:02<00:00, 545.81it/s]


判断是否存在云特征被重复检测的场景

In [9]:
cloud_infos['AVHRR_filename'] = cloud_infos['filepath'].apply(lambda x:os.path.basename(x))
unique_filename = np.unique(cloud_infos['AVHRR_filename'])
overfind = []
for i in tqdm(range(len(unique_filename))):
    temp_info = cloud_infos.loc[cloud_infos.AVHRR_filename==unique_filename[i]].copy()
    all_index = temp_info.index
    while len(all_index)>1:
        central_lon,central_lat = temp_info.loc[all_index[0],['cloud_lon','cloud_lat']].values
        #小于50KM时认为是一个,cluster至少含有一个点就是自己
        distancess = geodistance(central_lon,central_lat,temp_info.loc[all_index,'cloud_lon'].values,temp_info.loc[all_index,'cloud_lat'].values)
        cluster = temp_info.loc[all_index[distancess<=50]]
        all_index = all_index[distancess>50]
        if len(cluster)>1:
            overfind.append(cluster)
print(len(overfind))

100%|██████████| 15046/15046 [00:15<00:00, 953.02it/s] 

0


In [10]:
#统计每个ID的cloud时间
IDs = cloud_infos.ID.unique()
for i in range(len(IDs)):
    chosed_cloud = cloud_infos.loc[cloud_infos['ID']==IDs[i]]
    durhour = (chosed_cloud.mean_time.max()-chosed_cloud.mean_time.min()).total_seconds()/3600
    cloud_num = len(chosed_cloud)
    if durhour<4:
        print('ID:%d 持续时间%.2fh'%(IDs[i],durhour))
    if cloud_num<4:
        print('ID:%d 气旋%d个'%(IDs[i],cloud_num))

In [11]:
#气旋中心精确时间计算
cloud_times = []
for index,cloud_info in tqdm(cloud_infos.iterrows()):
    cloud_times.append(calculate_meantime(cloud_info.filepath,cloud_info.tle_name,cloud_info.cloud_lon,cloud_info.cloud_lat))
cloud_infos['cloud_time'] = cloud_times  

16418it [1:48:42,  2.52it/s]


同化其他信息

In [12]:
track_info_folder = r'E:\final_tracks\tracks_monthly_area'
track_infos = pd.concat([pd.read_excel(os.path.join(track_info_folder,i)).drop(columns=['Unnamed: 0','peak_value']) for i in os.listdir(track_info_folder)])
#一些错误边界的轨迹号
error_split_path_index = pd.read_csv(r"D:\Desktop\IMPMCT\error_area.txt",header=None).loc[:,0].values
ERA5_tracks_infos = pd.merge(ERA5_track,track_infos,on=['point_index','lon','lat','time','split_path_index','path_index'],how='inner')
error_area_bool = np.isin(ERA5_tracks_infos.split_path_index,error_split_path_index)
ERA5_tracks_infos.loc[error_area_bool,'isolated_area_path'] = ERA5_tracks_infos.loc[error_area_bool,'vort_min_line']

按照起始时间重排列ID

In [13]:
ID_dict = ERA5_tracks_infos.groupby('ID')['time'].min().reset_index().sort_values(['time'])
ID_dict['new_ID'] = np.arange(len(ID_dict))+1
ERA5_tracks_infos['ID'] = ERA5_tracks_infos['ID'].replace(ID_dict.ID.values,ID_dict.new_ID.values)
cloud_infos['ID'] = cloud_infos['ID'].replace(ID_dict.ID.values,ID_dict.new_ID.values)
cloud_infos = cloud_infos.sort_values(by=['ID','cloud_time']).reset_index(drop=True)
ERA5_tracks_infos = ERA5_tracks_infos.sort_values(by=['ID','time']).reset_index(drop=True)

In [14]:
#当每一个轨迹开始时，有邻接的涡旋认为是分裂
#当每一个轨迹结束时，有邻接的涡旋认为是合并
track_start_indexs = ERA5_tracks_infos.groupby('ID')['time'].idxmin()
track_start = ERA5_tracks_infos.loc[track_start_indexs]
split_IDs = track_start.loc[track_start.isolated_area_path!=track_start.original_vort_line]
#-------------------------------------------------------------------------------------------
track_end_indexs = ERA5_tracks_infos.groupby('ID')['time'].idxmax()
track_end = ERA5_tracks_infos.loc[track_end_indexs]
merge_IDs = track_end.loc[track_end.isolated_area_path!=track_end.original_vort_line]

In [15]:
#保存分裂与合并的ID
np.savetxt(r'D:\Desktop\final_dataset\IMPMCT\auxiliary information\split_IDs.txt', split_IDs.ID.values.reshape(-1, 1), fmt='%d')
np.savetxt(r'D:\Desktop\final_dataset\IMPMCT\auxiliary information\merge_IDs.txt', merge_IDs.ID.values.reshape(-1, 1), fmt='%d')

In [16]:
ERA5_tracks_infos.isolated_area_path = [np.array(ast.literal_eval(line)) for line in ERA5_tracks_infos.isolated_area_path]
ERA5_tracks_infos.original_vort_line = [np.array(ast.literal_eval(line)) for line in ERA5_tracks_infos.original_vort_line]
record_line_area_r = []
for isolated_area_path in tqdm(ERA5_tracks_infos.isolated_area_path.values):
    line_area  = area({'type':'Polygon','coordinates':[isolated_area_path.tolist()]})/1e6########   闭环,平方公里[(纬度，经度)]
    line_area_r = np.sqrt(line_area/np.pi)#涡旋的半径是由相同面积的圆的的半径
    record_line_area_r.append(line_area_r)
ERA5_tracks_infos['r'] = record_line_area_r


100%|██████████| 53787/53787 [00:01<00:00, 27308.01it/s]


是否有重叠的轨迹

In [17]:
ERA5_tracks_infos.value_counts(['time','slp_lon','slp_lat'])

time                 slp_lon  slp_lat
2001-02-20 05:00:00  -2.25    70.25      1
2014-02-28 18:00:00   48.25   79.50      1
2014-02-28 16:00:00   48.50   79.75      1
2014-02-28 15:00:00   48.75   79.75      1
2014-02-28 14:00:00   49.00   79.75      1
                                        ..
2008-03-15 01:00:00   5.25    75.50      1
2008-03-15 00:00:00   5.25    75.50      1
2008-03-14 23:00:00   4.75    75.75      1
2008-03-12 11:00:00  -22.50   59.50      1
2023-11-07 06:00:00   51.50   83.75      1
Name: count, Length: 33403, dtype: int64

气旋大小特征

In [18]:
#根据坐标计算点到直线距离
def get_distance_from_point_to_line(x0,y0, line):
    A = line[:,3]-line[:,1]#A=Y2-Y1
    B = line[:,0]-line[:,2]#B=X1-X2
    C = -(A * line[:,0] + B * line[:,1])#C=X1*(Y1-Y2)+Y1*(X2-X1)
    #根据点到直线的距离公式计算距离
    distance = np.abs(A * x0 + B * y0 + C) / (np.sqrt(A**2 + B**2))
    return distance

In [19]:
#长度和宽度,x1-x4,y1,y4逆时针旋转
w = np.sqrt((cloud_infos.x1-cloud_infos.x2)**2+(cloud_infos.y1-cloud_infos.y2)**2).values
h= np.sqrt((cloud_infos.x1-cloud_infos.x4)**2+(cloud_infos.y1-cloud_infos.y4)**2).values
r = np.concatenate([h.reshape((-1,1)),w.reshape((-1,1))],axis=1)
#width和length分别为矩形短边和长边
cloud_infos['width'],cloud_infos['length'] = np.min(r,axis=1)*2,np.max(r,axis=1)*2#KM

In [20]:
#计算云中心至短边的长距离和短距离
height_short_bool,width_short_bool= cloud_infos['width']==h*2,cloud_infos['width']==w*2
short_line1 = cloud_infos.loc[height_short_bool,['x2','y2','x3','y3']].values
short_line2 = cloud_infos.loc[height_short_bool,['x1','y1','x4','y4']].values
short_line3 = cloud_infos.loc[width_short_bool,['x1','y1','x2','y2']].values
short_line4 = cloud_infos.loc[width_short_bool,['x4','y4','x3','y3']].values
dis1 = get_distance_from_point_to_line(cloud_infos.loc[height_short_bool,'x0'].values,cloud_infos.loc[height_short_bool,'y0'].values, short_line1)
dis2 = get_distance_from_point_to_line(cloud_infos.loc[height_short_bool,'x0'].values,cloud_infos.loc[height_short_bool,'y0'].values, short_line2)
dis3 = get_distance_from_point_to_line(cloud_infos.loc[width_short_bool,'x0'].values,cloud_infos.loc[width_short_bool,'y0'].values, short_line3)
dis4 = get_distance_from_point_to_line(cloud_infos.loc[width_short_bool,'x0'].values,cloud_infos.loc[width_short_bool,'y0'].values, short_line4)
cloud_infos.loc[height_short_bool,'dis1'] = dis1
cloud_infos.loc[height_short_bool,'dis2'] = dis2
cloud_infos.loc[width_short_bool,'dis1'] = dis3
cloud_infos.loc[width_short_bool,'dis2'] = dis4
cloud_infos['l_search'] = np.max(cloud_infos[['dis1','dis2']],axis=1)*2
cloud_infos['s_search'] = np.min(cloud_infos[['dis1','dis2']],axis=1)*2

In [21]:
step_describe = cloud_infos.value_counts('ID').reset_index().sort_values('ID')
steps = np.concatenate(([np.arange(count)+1 for count in step_describe['count'].values]))
cloud_infos['step'] = steps
cloud_infos.drop(columns=['is_visible','dis1','dis2']).to_excel(r'D:\Desktop\final_dataset\cloud_infos.xlsx')
ERA5_tracks_infos.to_excel(r'D:\Desktop\final_dataset\tracks.xlsx')

In [22]:
cloud_infos.drop(columns=['is_visible','dis1','dis2']).to_excel(r'D:\Desktop\final_dataset\cloud_infos.xlsx')

保存数据集

In [23]:
IMPMCT_tracks  = ERA5_tracks_infos.rename({'peak_value':'ζ_smth850'},axis=1)
new_columns_order  = ['ID','time','lon','lat','ζ_smth850','r','slp_lon','slp_lat','slp']
IMPMCT_tracks = IMPMCT_tracks[new_columns_order].reset_index(drop=True)
IMPMCT_tracks[['ζ_smth850', 'r', 'slp_lon','slp']] = np.round(IMPMCT_tracks[['ζ_smth850', 'r', 'slp_lon','slp']],2)
IMPMCT_tracks.to_excel(r"D:\Desktop\final_dataset\IMPMCT\ERA5_tracks.xlsx",index=None)

处理隔一个小时连接的轨迹

In [24]:
ERA5_tracks = pd.read_excel(r'D:\Desktop\final_dataset\IMPMCT\ERA5_tracks.xlsx')
ERA5_tracks

,ID,time,lon,lat,ζ_smth850,r,slp_lon,slp_lat,slp
0,1,2001-02-20 02:00:00,-3.50,68.50,1.27,50.26,NaN,NaN,NaN
1,1,2001-02-20 03:00:00,-1.75,68.50,1.37,81.97,NaN,NaN,NaN
2,1,2001-02-20 04:00:00,-0.75,68.75,1.47,97.21,NaN,NaN,NaN
3,1,2001-02-20 05:00:00,-0.25,69.50,1.65,113.22,-2.25,70.25,988.02
4,1,2001-02-20 06:00:00,1.50,69.50,1.76,124.18,-0.75,70.25,986.56
...,...,...,...,...,...,...,...,...,...
53782,1185,2023-11-07 02:00:00,51.50,83.25,2.32,102.30,52.25,83.25,1002.35
53783,1185,2023-11-07 03:00:00,51.50,83.50,2.30,103.10,52.25,83.25,1002.51
53784,1185,2023-11-07 04:00:00,51.00,83.75,2.22,103.82,52.25,83.50,1003.06
53785,1185,2023-11-07 05:00:00,50.75,84.00,2.10,103.44,52.00,83.75,1003.23


In [25]:
record = []
for ID in ERA5_tracks.ID.unique():
    chosed_track = ERA5_tracks.loc[ERA5_tracks.ID==ID]
    chosed_record = [chosed_track]
    for j in range(len(chosed_track)-1):
        delta_hour = (chosed_track.iloc[j+1]['time']-chosed_track.iloc[j]['time']).total_seconds()/3600
        if delta_hour==2:
            new_info = chosed_track.iloc[j:j+1].copy()
            new_info[['slp_lon','slp_lat','slp']] = np.nan
            new_info['time'] = chosed_track.iloc[j]['time']+timedelta(hours=1)
            new_info[['lon','lat','ζ_smth850','r']] = np.mean(chosed_track.iloc[j:j+2][['lon','lat','ζ_smth850','r']],axis=0)
            chosed_record.append(new_info)
    record.append(pd.concat(chosed_record,axis=0))

In [26]:
ERA5_tracks = pd.concat(record,axis=0).sort_values(['ID','time'])
ERA5_tracks

,ID,time,lon,lat,ζ_smth850,r,slp_lon,slp_lat,slp
0,1,2001-02-20 02:00:00,-3.50,68.50,1.27,50.26,NaN,NaN,NaN
1,1,2001-02-20 03:00:00,-1.75,68.50,1.37,81.97,NaN,NaN,NaN
2,1,2001-02-20 04:00:00,-0.75,68.75,1.47,97.21,NaN,NaN,NaN
3,1,2001-02-20 05:00:00,-0.25,69.50,1.65,113.22,-2.25,70.25,988.02
4,1,2001-02-20 06:00:00,1.50,69.50,1.76,124.18,-0.75,70.25,986.56
...,...,...,...,...,...,...,...,...,...
53782,1185,2023-11-07 02:00:00,51.50,83.25,2.32,102.30,52.25,83.25,1002.35
53783,1185,2023-11-07 03:00:00,51.50,83.50,2.30,103.10,52.25,83.25,1002.51
53784,1185,2023-11-07 04:00:00,51.00,83.75,2.22,103.82,52.25,83.50,1003.06
53785,1185,2023-11-07 05:00:00,50.75,84.00,2.10,103.44,52.00,83.75,1003.23


In [27]:
record = []
for ID in tqdm(ERA5_tracks.ID.unique()):
    chosed_track = ERA5_tracks.loc[ERA5_tracks.ID==ID]
    chosed_record = [chosed_track]
    for j in range(len(chosed_track)-1):
        delta_hour = (chosed_track.iloc[j+1]['time']-chosed_track.iloc[j]['time']).total_seconds()/3600
        if delta_hour>1:
            print(ID)


 38%|███▊      | 447/1185 [00:04<00:07, 104.31it/s]

436


100%|██████████| 1185/1185 [00:12<00:00, 95.62it/s] 


In [28]:
print('ID为436的轨迹相差3个时次,其在云图中连接,在ERA5中断开')

ID为436的轨迹相差3个时次,其在云图中连接,在ERA5中断开


In [29]:
ERA5_tracks.to_excel(r'D:\Desktop\final_dataset\IMPMCT\ERA5_tracks.xlsx',index=None)